# Data science approach to organizing my playlist
[Complete Post](https://arjon.es/2020/data-science-approach-to-organizing-my-playlist/)

## Model constraint solving

In [1]:
import numpy as np
import pandas as pd

raw = pd.read_csv('my-playlist.csv')

In [2]:
COLUMNS_REMOVE = ['liveness', 'acousticness', 'speechiness', 'loudness', 'instrumentalness', 'valence', 'tempo']
df = raw.drop(columns=COLUMNS_REMOVE) #[COLUMNS_KEEP]

# Create score field to order songs
df['score'] = 0

In [3]:
def filter_field(column, df):
    return df[(df[column]==df[column].min()) | (df[column]==df[column].max())].sample(n=1, random_state=42)

In [4]:
frames = []

for column in df.loc[:, df.dtypes == np.float64].columns:
    frames.append(filter_field(column, df))

data = pd.concat(frames).drop_duplicates()

In [5]:
dataset_size = len(df)
desired_sample = int(dataset_size * 0.055)
current_sample = len(data)

if desired_sample > current_sample:
    n = (desired_sample - current_sample)
    data = data.append(df.sample(n=n, random_state=42)).drop_duplicates()

In [6]:
data['url'] = data[['track_id']].apply(lambda x: f"https://open.spotify.com/track/{x['track_id']}", axis=1)

In [7]:
def ddisplay(df):
    def make_clickable(url):
        return f'<a target="_blank" href="{url}">listen</a>'

    display(df.style.format({'url': make_clickable}))
    pass

ddisplay(data)

,track_id,track_name,danceability,energy,score,url
89,5VwQYHpyQPhiToPKWJaHiO,I Can't Dance,0.941000,0.497000,0,listen
155,2wPFy7SAFnt9Nj2TipWcqb,Block Rockin' Beats,0.606000,0.977000,0,listen
281,6IvKz4Ax11LsLA0X0bPkqD,"Ai, Ai, Ai...",0.643000,0.673000,0,listen
265,53eJFr4Mfbw5PXJ01K6cFw,Daughter (Remastered),0.615000,0.702000,0,listen
164,4tb7A6O8cE2UzELRTwU3KZ,Só Tinha De Ser Com Você,0.807000,0.476000,0,listen
9,2kMvM1l45ZjpOv73qPefxA,Relicário,0.460000,0.515000,0,listen
77,7C1bOiV8Bj0I3YpAnuS8ue,São Gonça,0.639000,0.777000,0,listen
278,1FZ90IoWFZlubV6bjyyEwq,Faded - Dash Berlin Remix,0.388000,0.915000,0,listen
93,1idZSAVHfLPQq9tDTYhnIA,The Last of the Famous International Playboys - 2010 Remaster,0.617000,0.876000,0,listen
109,5mnvqisoDJilY0uCEdT8rG,Danny's Song,0.507000,0.198000,0,listen


In [8]:
# Ordering songs
ORDER = [
    155, # Block Rockin' Beats
    278, # Faded - Dash Berlin Remix
    154, # Go
    97,  # Smooth
    89,  # I Can't Dance
    93,  # The Last of the Famous International Playboys
    265, # Daughter (Remastered)
    195, # Because the Night - MTV Unplugged Version
    173, # Crazy
    77,  # São Gonça
    109, # Danny's Song
    281, # Ai, Ai, Ai...
    184, # Segue o Som
    5,   # Loving Every Minute
    9,   # Relicário
    164, # Só Tinha De Ser Com Você
]

TOTAL = len(set(ORDER))

for idx, pandas_idx in enumerate(ORDER, start=1):
    data.at[pandas_idx, 'score'] = (1 + TOTAL - idx)

In [9]:
ordered_df = data.sort_values(by='score', ascending=False)
ordered_df

,track_id,track_name,danceability,energy,score,url
155,2wPFy7SAFnt9Nj2TipWcqb,Block Rockin' Beats,0.606,0.977,16,https://open.spotify.com/track/2wPFy7SAFnt9Nj2...
278,1FZ90IoWFZlubV6bjyyEwq,Faded - Dash Berlin Remix,0.388,0.915,15,https://open.spotify.com/track/1FZ90IoWFZlubV6...
154,2cNjgoSh1TBHFQIhfzRJUE,Go,0.751,0.897,14,https://open.spotify.com/track/2cNjgoSh1TBHFQI...
97,0n2SEXB2qoRQg171q7XqeW,Smooth (feat. Rob Thomas),0.609,0.923,13,https://open.spotify.com/track/0n2SEXB2qoRQg17...
89,5VwQYHpyQPhiToPKWJaHiO,I Can't Dance,0.941,0.497,12,https://open.spotify.com/track/5VwQYHpyQPhiToP...
93,1idZSAVHfLPQq9tDTYhnIA,The Last of the Famous International Playboys ...,0.617,0.876,11,https://open.spotify.com/track/1idZSAVHfLPQq9t...
265,53eJFr4Mfbw5PXJ01K6cFw,Daughter (Remastered),0.615,0.702,10,https://open.spotify.com/track/53eJFr4Mfbw5PXJ...
195,5VzvK7YiwWbRePA2JOzhYq,Because the Night - MTV Unplugged Version,0.408,0.790,9,https://open.spotify.com/track/5VzvK7YiwWbRePA...
173,6Vz7vzOpCwKeSQlfViibuY,Crazy,0.633,0.858,8,https://open.spotify.com/track/6Vz7vzOpCwKeSQl...
77,7C1bOiV8Bj0I3YpAnuS8ue,São Gonça,0.639,0.777,7,https://open.spotify.com/track/7C1bOiV8Bj0I3Yp...


In [10]:
exprs = []
for index, row in ordered_df.iterrows():
    expr = "(" + \
        f"danceability * {row['danceability']} + " + \
        f"energy * {row['energy']}" + \
        ")"
    exprs.append(expr)

In [11]:
for idx in range(len(exprs)-1):
    print(' > '.join(exprs[idx: idx+2]), ', ')

(danceability * 0.606 + energy * 0.977) > (danceability * 0.38799999999999996 + energy * 0.915) , 
(danceability * 0.38799999999999996 + energy * 0.915) > (danceability * 0.7509999999999999 + energy * 0.897) , 
(danceability * 0.7509999999999999 + energy * 0.897) > (danceability * 0.609 + energy * 0.9229999999999999) , 
(danceability * 0.609 + energy * 0.9229999999999999) > (danceability * 0.941 + energy * 0.49700000000000005) , 
(danceability * 0.941 + energy * 0.49700000000000005) > (danceability * 0.617 + energy * 0.8759999999999999) , 
(danceability * 0.617 + energy * 0.8759999999999999) > (danceability * 0.615 + energy * 0.7020000000000001) , 
(danceability * 0.615 + energy * 0.7020000000000001) > (danceability * 0.408 + energy * 0.79) , 
(danceability * 0.408 + energy * 0.79) > (danceability * 0.633 + energy * 0.858) , 
(danceability * 0.633 + energy * 0.858) > (danceability * 0.639 + energy * 0.777) , 
(danceability * 0.639 + energy * 0.777) > (danceability * 0.507 + energy * 0.

---
# Using Z3 to stablish order
[Z3Prover](https://github.com/Z3Prover/z3)

In [12]:
from z3 import *

In [13]:
# Define variables
danceability = Real('danceability')
energy = Real('energy')
# tempo = Real('tempo')
# loudness = Real('loudness')
# valence = Real('valence')
# speechiness = Real('speechiness')
# instrumentalness = Real('instrumentalness')
# liveness = Real('liveness')
# acousticness = Real('acousticness')

In [14]:
# Start Solver
s = Solver()

In [15]:
# Add Constraints
s.add(
(danceability * 0.606 + energy * 0.977) > (danceability * 0.38799999999999996 + energy * 0.915) , 
(danceability * 0.38799999999999996 + energy * 0.915) > (danceability * 0.7509999999999999 + energy * 0.897) , 
(danceability * 0.7509999999999999 + energy * 0.897) > (danceability * 0.609 + energy * 0.9229999999999999) , 
(danceability * 0.609 + energy * 0.9229999999999999) > (danceability * 0.941 + energy * 0.49700000000000005) , 
(danceability * 0.941 + energy * 0.49700000000000005) > (danceability * 0.617 + energy * 0.8759999999999999) , 
(danceability * 0.617 + energy * 0.8759999999999999) > (danceability * 0.615 + energy * 0.7020000000000001) , 
(danceability * 0.615 + energy * 0.7020000000000001) > (danceability * 0.408 + energy * 0.79) , 
(danceability * 0.408 + energy * 0.79) > (danceability * 0.633 + energy * 0.858) , 
(danceability * 0.633 + energy * 0.858) > (danceability * 0.639 + energy * 0.777) , 
(danceability * 0.639 + energy * 0.777) > (danceability * 0.507 + energy * 0.198) , 
(danceability * 0.507 + energy * 0.198) > (danceability * 0.643 + energy * 0.6729999999999999) , 
(danceability * 0.643 + energy * 0.6729999999999999) > (danceability * 0.7809999999999999 + energy * 0.544) , 
(danceability * 0.7809999999999999 + energy * 0.544) > (danceability * 0.557 + energy * 0.528) , 
(danceability * 0.557 + energy * 0.528) > (danceability * 0.46 + energy * 0.515) , 
(danceability * 0.46 + energy * 0.515) > (danceability * 0.807 + energy * 0.47600000000000003)
)

In [16]:
s.check()

unsat